# 실습 세션 2: AutoGen 기반 여행 플래너 Multi-Agent 앱 구현

이 세션에서는 Microsoft AutoGen 프레임워크를 활용하여 여행 플래너 멀티 에이전트 애플리케이션을 구현합니다. 다양한 역할의 AI 에이전트가 협업하여 사용자의 여행 계획을 도와주는 시스템을 만들어봅니다.

## 1. GitHub Copilot 활성화 및 Agent Mode 소개

이 실습에서는 GitHub Copilot의 Agent Mode를 활용합니다. 아래 내용을 참고하여 Copilot을 활성화하고, Edit/Chat 모드의 차이와 사용법을 익히세요.

### GitHub Copilot 활성화 방법
1. VS Code에서 [Extensions]에서 'GitHub Copilot'을 설치합니다.
2. 상태바 또는 Command Palette(Cmd+Shift+P)에서 'Copilot: Sign In'을 선택해 GitHub 계정으로 로그인합니다.
3. Copilot이 정상적으로 활성화되면, 에디터 우측 하단에 Copilot 아이콘이 표시됩니다.

### Agent Mode란?
- **Agent Mode**는 Copilot이 단순 코드 자동완성(Inline) 기능을 넘어, 파일 편집(Edit)과 대화(Chat) 기반의 다양한 개발 작업을 지원하는 모드입니다.
- Edit/Chat 모드 모두 Agent Mode의 일부로, 더 복잡한 작업을 Copilot이 직접 수행할 수 있습니다.

#### Edit 모드
- 코드 파일에서 특정 영역을 선택한 뒤, Copilot 패널에서 "Edit with Copilot"을 클릭하면 선택 영역을 수정/리팩터링/설명/변환 등 다양한 작업을 요청할 수 있습니다.
- 예시: 함수 블록을 선택 → "함수에 타입 힌트 추가" 요청

#### Chat 모드
- Copilot Chat 패널(사이드바 또는 하단)에서 자연어로 질문하거나, 파일/코드/작업에 대한 지시를 내릴 수 있습니다.
- 예시: "이 파일의 주요 버그를 찾아줘", "이 함수의 시간복잡도를 설명해줘"

> **Tip:** 실습 중 코드 셀이나 파일에서 Copilot Chat에 `/edit` 명령어를 입력하면, 선택 영역을 Edit 모드로 바로 전환할 수 있습니다.

---

## 2. 필요한 라이브러리 설치

AutoGen 및 관련 라이브러리를 설치합니다.

In [18]:
# AutoGen 및 필요한 라이브러리 설치
!pip install -r requirements.txt

## 3. AutoGen 프레임워크 및 환경 설정

OpenAI API 키 등 환경 변수를 설정하고, 모델 클라이언트를 준비합니다.

In [19]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), # For key-based authentication.
)

print("AutoGen 설정이 완료되었습니다.")

AutoGen 설정이 완료되었습니다.


## 3. 여행 플래너 Multi-Agent 시스템 구성

여행 플래너, 로컬 추천, 언어 팁, 여행 요약 등 다양한 역할의 에이전트를 정의합니다.

In [20]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio

planner_agent = AssistantAgent(
    name="planner_agent",
    description="A helpful assistant that can plan trips",
    model_client=model_client,
    system_message="""You are a helpful assistant that can suggest a travel plan for a user based on their request.\nEnd your planning with 'INITIAL PLAN COMPLETE'."""
)

local_agent = AssistantAgent(
    name="local_agent",
    description="A local assistant that can suggest local activities or places to visit",
    model_client=model_client,
    system_message="""You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user.\nEnd your recommendations with 'LOCAL RECOMMENDATIONS COMPLETE'."""
)

language_agent = AssistantAgent(
    name="language_agent",
    description="A helpful assistant that can provide language tips for a given destination",
    model_client=model_client,
    system_message="""You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination.\nEnd your language tips with 'LANGUAGE TIPS COMPLETE'."""
)

travel_summary_agent = AssistantAgent(
    name="travel_summary_agent",
    description="A helpful assistant that can summarize the travel plan",
    model_client=model_client,
    system_message="""You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan.\nYOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN.\nWhen the plan is complete and all perspectives are integrated, you can respond with TERMINATE."""
)

user_proxy_travel = UserProxyAgent(
    name="user_proxy",
    description="A traveler seeking travel planning assistance"
)

## 4. 그룹 채팅 및 종료 조건 설정

RoundRobinGroupChat을 사용하여 에이전트 협업 구조를 만들고, 대화 종료 조건을 정의합니다.

In [21]:
termination = MaxMessageTermination(max_messages=30) | TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent, user_proxy_travel],
    termination_condition=termination
)

## 5. 여행 플래너 Multi-Agent 대화 실행

아래 코드를 실행하면, 다양한 에이전트가 협업하여 여행 계획을 완성합니다.

In [23]:
travel_task = """I'm planning a 5-day trip to Paris with my friend. We love art, food, and want to experience both famous sights and some local hidden gems. We don't speak French. Please help create a detailed travel plan."""

async def run_travel_planner():
    await Console(group_chat.run_stream(task=travel_task))

# 실행:
await run_travel_planner()

---------- TextMessage (user) ----------
I'm planning a 5-day trip to Paris with my friend. We love art, food, and want to experience both famous sights and some local hidden gems. We don't speak French. Please help create a detailed travel plan.
---------- TextMessage (planner_agent) ----------
Here’s a 5-day travel plan for your trip to Paris that balances well-known attractions with local discoveries while catering to your love for art and food.

### Day 1: Arrival and Montmartre Exploration
- **Morning:** Arrive in Paris. Check into your accommodation (consider staying in the Montmartre district for an artistic vibe).
- **Lunch:** Grab a traditional French lunch at *Le Consulat* or *La Crêperie de Josselin*. Try a savory galette or a classic croque monsieur.
- **Afternoon:** Explore the charming streets of Montmartre. Visit:
  - **Sacré-Cœur Basilica:** Climb to the dome for stunning views of the city.
  - **Place du Tertre:** Watch local artists at work and consider getting a port

## 6. AutoGen Studio 실행 가이드

AutoGen Studio는 웹 기반의 시각적 인터페이스로, 다양한 에이전트 시나리오를 손쉽게 실험하고 관리할 수 있습니다.

- 설치: 터미널에서 `pip install autogenstudio` 실행
- 실행: 터미널에서 `autogenstudio` 명령어 입력
- 웹 브라우저에서 http://localhost:8080 으로 접속
- 다양한 프롬프트, 에이전트, 팀 시나리오를 GUI로 실험 가능

> **참고:** AutoGen Studio는 Python 3.10+ 환경에서 권장되며, 일부 기능은 최신 AutoGen 버전에서만 지원될 수 있습니다.

## 7. 마무리 및 다음 단계

이 세션에서는 AutoGen을 활용한 여행 플래너 Multi-Agent 시스템을 구현했습니다. 다양한 역할의 에이전트가 협업하여 사용자의 요구에 맞는 여행 일정을 자동으로 생성하는 과정을 실습했습니다.

- 다양한 역할의 에이전트 설계 및 협업 구조 이해
- SelectorGroupChat을 통한 동적 역할 분배
- 실전 여행 플래너 예제 구현

더 많은 예제와 심화 내용은 [AutoGen 공식 문서](https://microsoft.github.io/autogen/)를 참고하세요.